In [45]:
base_path="../input/"
train_file = base_path+"train_v2.csv"
test_file = base_path+"test_v2.csv"
my_submission_file = base_path+"mysub.csv"

## Generic

In [46]:
#mathplotlib
import matplotlib.pyplot as plt
from matplotlib import pylab
import matplotlib as mpl
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt

#math
import math

#numpy
from numpy import arange,array,ones
import numpy as np

#scipy
from scipy import stats
import scipy

#csv
import csv, json

#pandas
import pandas as pd
import pandas
from pandas.plotting import scatter_matrix
from pandas.io.json import json_normalize

#seaborn
import seaborn as sns

#options
get_ipython().run_line_magic('matplotlib', 'notebook')
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

import warnings
warnings.simplefilter("ignore")

#lgb
import lightgbm as lgb

#sklearn
from sklearn import model_selection
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn import preprocessing
from sklearn import metrics
from sklearn.preprocessing import OneHotEncoder

#os
import os
import datetime
import sys, time

def common_feats(train, test):
    return list(set(test.columns).intersection(set(train.columns)))


def drop_uncommons(train, test, exceptions):
    commons = common_feats(train, test)

    for col in train.columns:
        if (col not in commons) & (col not in exceptions):
            train = train.drop(col, axis=1)

    for col in test.columns:
        if (col not in commons) & (col not in exceptions):
            test = test.drop(col, axis=1)

    return train, test


class toolbar:

    def __init__(self, length=10):
        self.length = length
        self.steps = 1

    def set_len(self, new):
        self.length = new

    def prin(self):
        if (self.steps <= self.length):
            str1 = "["
            for i in range(self.steps):
                str1 += '*'
            for i in range(self.length - self.steps):
                str1 += ' '
            str1 += "]"
            sys.stdout.write('\r' * self.length + str(str1))

            self.steps += 1

    def update(self):
        if (self.steps < self.length):
            self.steps += 1


#tipi numerici
tipi_num = [np.int64,float,int]

def find_num_cols(df,exceptions,cats,nums):
    vet = []
    for col in df.columns:
        try:
            if (type(eval(df[col][0])) in tipi_num) & (col not in cats) & (col not in nums) & (
                    df[col].nunique() > 10) & (col not in exceptions):
                #print("aggiugngo ai num_cols", col)
                vet += [col]
        except:
            try:
                if (type(df[col][0]) in tipi_num) & (col not in cats) & (col not in nums) & (
                        df[col].nunique() > 10) & (col not in exceptions):
                    #print("aggiugngo ai num_cols", col)
                    vet += [col]
            except:
                pass

    vet += nums
    return vet


def find_cat_cols(df,exceptions,nums):
    return list( (set(df.columns).difference(set(nums))).difference(set(exceptions))   )

class game:

    def __init__(self):
        self.ph = pd.DataFrame([["Florida", 2], ["Florida", 8], ["Arizona", 4]], columns=["a", "b"])

    def view(self):
        self.ph.head()

    def applica(self, fun):
        return fun(self.ph)

    def restore(self):
        self.ph = pd.DataFrame([["Florida", 2], ["Florida", 8], ["Arizona", 4]], columns=["a", "b"])
        return self.ph

    def mean(self):
        return self.ph.groupby("a").mean()

    def get(self):
        return self.ph
        
def stringify_cats(df,cats):
    df[cats] = df[cats].astype(str)
    return df

def drop_exceeding(train,test,max_new_feat,cats,target):
    da_droppare=[]

    for col in cats:
        if (train[col].nunique()>max_new_feat) & (col not in target):
            da_droppare+=[col]
            train=train.drop(col,axis=1)

    for col in da_droppare:
        test=test.drop(col,axis=1)

    new=[]
    for col in cats:
        if col not in da_droppare:
            new += [col]

    cats = new
    
    return train,test,cats

# check_coherence between train_df cat_cols num_cols test_df
def cc(train_df,test_df,cats,nums):
    if len(list(set(cats).intersection(set(nums)))) > 0:
        raise Exception("intersection not empty between cats & nums")

    for col in cats:
        if col not in train_df.columns:
            raise Exception("incohererence between cat_cols and train_df.columns with col {}".format(col))

    for col in nums:
        if col not in train_df.columns:
            raise Exception("incohererence between num_cols and train_df.columns with col {}".format(col))

    for col in cats:
        if col not in test_df.columns:
            raise Exception("incohererence between cat_cols and test_df.columns with col {}".format(col))

    for col in nums:
        if (col not in test_df.columns) & (col != "totals.totalTransactionRevenue"):
            raise Exception("incohererence between num_cols and test_df.columns with col {}".format(col))
            
    if "totals.totalTransactionRevenue" not in nums:
        raise Exception("incohererence: nums is corrupted")

    if "totals.totalTransactionRevenue" not in train_df.columns:
        raise Exception("incohererence: transactionRevenue not in train_df.columns")

    if ("fullVisitorId" not in train_df.columns) | ("fullVisitorId" not in test_df.columns):
        raise Exception("incohererence: fullVisitorId not in train_df.columns or test_df.columns")
        
    if len(list(set(cats).intersection(set(nums)))):
        raise Exception("intersection not-empty")
        
    for col in train_df.columns:
        if type(col)!=type(""):
            raise Exception("uncorrect type in cols names of train")
            
    for col in test_df.columns:
        if type(col)!=type(""):
            raise Exception("uncorrect type in cols names of test")
            
    if "totals.totalTransactionRevenue" in test_df.columns:
        raise Exception("target dont have to stay in test dataset")
        
        
        
def norm(l):
    return (l-min(l))/(max(l) - min(l))

## Preprocessing

In [47]:
#mathplotlib
import matplotlib.pyplot as plt
from matplotlib import pylab
import matplotlib as mpl
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt

#math
import math

#numpy
from numpy import arange,array,ones
import numpy as np

#scipy
from scipy import stats
import scipy

#csv
import csv, json

#pandas
import pandas as pd
import pandas
from pandas.plotting import scatter_matrix
from pandas.io.json import json_normalize

#seaborn
import seaborn as sns

#options
get_ipython().run_line_magic('matplotlib', 'notebook')
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

import warnings
warnings.simplefilter("ignore")

#lgb
import lightgbm as lgb

#sklearn
from sklearn import model_selection
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn import preprocessing
from sklearn import metrics
from sklearn.preprocessing import OneHotEncoder

#os
import os
import datetime
import sys, time

def flatten(twoDim):
    support = []
    for new_feats in twoDim:
        for feat in new_feats:
            support += [feat]
    return support

def into_dataframe(col_names, matr):
    new_data = pd.DataFrame()
    for i, name in enumerate(col_names):
        new_data[name] = pd.Series(matr[:, i])

    return new_data

def drop_cols(df, not_drop):
    return df[not_drop]

def encode_cats(train, test, cats):
    t = toolbar(11)

    t.prin()
    # creo encoder
    oh_enc = OneHotEncoder(sparse=False)

    t.prin()
    # encodo e prendo i nomi vecchi
    new_cols_train = oh_enc.fit_transform(array(train[cats]))
    feat_train = oh_enc.categories_

    t.prin()
    # anche per il test
    new_cols_test = oh_enc.fit_transform(array(test[cats]))
    feat_test = oh_enc.categories_

    t.prin()
    # i nomi vecchi sono una matr n*n, la rendo n*1
    feat_train = flatten(feat_train)

    # anche per il test
    feat_test = flatten(feat_test)

    t.prin()
    # trasformo le nuove colonne in un dataframe
    new_train = into_dataframe(feat_train, new_cols_train)

    t.prin()
    # anche per il test
    new_test = into_dataframe(feat_test, new_cols_test)

    t.prin()
    # cerco le colonne uguali nei dataframes
    commons = common_feats(new_train, new_test)

    t.prin()
    new_train = drop_cols(new_train, commons)
    new_test = drop_cols(new_test, commons)

    t.prin()
    for col in cats:
        train = train.drop(col, axis=1)
        test = test.drop(col, axis=1)

    t.prin()
    cats = commons

    t.prin()
    train = train.merge(new_train, right_index=True, left_index=True)
    test = test.merge(new_test, right_index=True, left_index=True)

    t.prin()
    return train, test, cats

# In[70]:


def reset_index(df, groupper):
    df[groupper] = df.index
    df = df.set_index(array([x for x in range(df.shape[0])]))

    return df


def group_me(df,groupper,cats,nums,mode):
    t = toolbar(6)

    t.prin()
    # raggruppo per il valore groupper

    # raccolgo con la media per i numeri
    t.prin()
    df_num = pd.DataFrame()
    
    train=0
    
    try:
        if "totals.totalTransactionRevenue" in df.columns:
            train=1
    except:
        pass
    
    if train==1:
        #siamo nel train
        #print(set(nums).difference(set(nums).intersection(set(df.columns))))
        df_num[nums] = df.groupby(groupper)[nums].mean()
    else:
        #siamo nel test
        nn = list(set(nums).difference(set(["totals.totalTransactionRevenue"])))
        df_num[nn] = df.groupby(groupper)[nn].mean()
        

    # raggruppo per la mediana per le categorie
    t.prin()
    df_cat = pd.DataFrame()
    groupped = df.groupby(groupper)
    
    if mode == "mode":
        for col in cats:
            df_cat[col] = groupped[col].agg(lambda x: stats.mode(x)[0][0])
    if mode == "mean":
        df_cat[cats] = df.groupby(groupper)[cats].mean()
    if mode == "median":
        df_cat[cats] = df.groupby(groupper)[cats].median()
    if mode == "mode_approx":
        my_group = df.groupby(groupper)[cats]
        df_cat[cats] = -(2*my_group.mean() - 3*my_group.median())
        

    t.prin()
    # unisco i due nuovi dataframes
    df = df_num.merge(df_cat, right_index=True, left_index=True)

    t.prin()
    # df[groupper]=df.index
    # df = df.set_index(array([x for x in range(df.shape[0])]))
    df = reset_index(df, groupper)

    t.prin()
    return df

## Linear Regression

In [48]:
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LinearRegression
import numpy as np
from sklearn.preprocessing import MinMaxScaler

def norm(df):
    return (df-df.min())/(df.max()-df.min())

def lin(dev,val,test):
    scaler = MinMaxScaler()
    dev = scaler.fit_transform(dev)
    test = scaler.fit_transform(test)

    lr = LinearRegression().fit(np.array(norm(dev)), np.array(val))
    predictions = lr.predict(np.array(test))
    predictions[predictions<0] = 0
    
    return predictions

## Dataset Parsing

In [ ]:
#mathplotlib
import matplotlib.pyplot as plt
from matplotlib import pylab
import matplotlib as mpl
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt

#math
import math

#numpy
from numpy import arange,array,ones
import numpy as np

#scipy
from scipy import stats
import scipy

#csv
import csv, json

#pandas
import pandas as pd
from pandas.plotting import scatter_matrix
from pandas.io.json import json_normalize

#seaborn
import seaborn as sns

#options
get_ipython().run_line_magic('matplotlib', 'notebook')
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

import warnings
warnings.simplefilter("ignore")

#lgb
import lightgbm as lgb

#sklearn
from sklearn import model_selection
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn import preprocessing
from sklearn import metrics
from sklearn.preprocessing import OneHotEncoder

#os
import os
import datetime
import sys, time

# trasforma liste di dizionari in un solo dizionario
def unpack(col):
    mod = []
    for row in col:
        try:
            mod += eval(row)
        except:
            mod += row

    return pd.Series(mod)


def correction_nan(x):
    if type(x) == type(0.0):
        return {}
    else:
        return x


def mappa(fun, ite):
    return [fun(x) for x in ite]


# è una sola la colonna che ricevo e la trasformo in un dict
def from_dict_to_frame(df, col):
    # return pd.DataFrame(df[col])

    df[col] = mappa(correction_nan, df[col])

    new = []
    for c in df[col]:
        new += [c]
    new2 = pd.DataFrame.from_records(new)
    # corretto new2 -> print(type(new2))

    return new2


# trasforma n colonne con un dict all interno in un dizionario che le comprende
def unfold(df, cols,target):
    for col in cols:
        if col not in target:
            df = df.merge(from_dict_to_frame(df, col), right_index=True, left_index=True)
            df = df.drop(col, axis=1)

    return df


# trasforma jsons in colonne
def load_jsons_as_cols(df, JSON_COLUMNS):
    for column in JSON_COLUMNS:
        column_as_df = json_normalize(df[column])
        column_as_df.columns = [f"{column}.{subcolumn}" for subcolumn in column_as_df.columns]
        df = df.drop(column, axis=1).merge(column_as_df, right_index=True, left_index=True)

    return df


def find_dicts(df,target):
    dicts_to_dump = []
    for x in range(len(df.columns)):
        try:
            if (type(df.iloc[0, x]) == type({})) & (df.columns[x] not in target):
                dicts_to_dump += [df.columns[x]]
        except:
            try:
                if (type(eval(df.iloc[0, x])) == type({})) & (df.columns[x] not in target):
                    dicts_to_dump += [df.columns[x]]
            except:
                pass
    return dicts_to_dump


def find_lists(df,target):
    lists_to_dump = []
    for x in range(len(df.columns)):
        try:
            if (type(eval(df.iloc[0, x])) == type([])) & (df.columns[x] not in target):
                lists_to_dump += [df.columns[x]]
        except:
            try:
                if (type(df.iloc[0, x]) == type([])) & (df.columns[x] not in target):
                    lists_to_dump += [df.columns[x]]
            except:
                pass

    return lists_to_dump


def drop_const(df,target):
    for col in df.columns:
        if (df[col].astype(str).nunique(dropna=False) == 1) & (col not in target):
            df = df.drop(col, axis=1)

    return df


def drop_uniques(df,target):
    for col in df.columns:
        if (df[col].astype(str).nunique(dropna=False) == df.shape[0]) & (col not in target):
            df = df.drop(col, axis=1)

    return df

def load_df(csv_path,n_rows,target):
    JSON_COLUMNS = ["device", "geoNetwork", "totals", "trafficSource"]

    df = pd.DataFrame()

    if n_rows == -1:
        df = pd.read_csv(csv_path,
                         converters={column: json.loads for column in JSON_COLUMNS}
                         , dtype={"fullVisitorId": "str"}
                         )
    else:
        df = pd.read_csv(csv_path,
                         converters={column: json.loads for column in JSON_COLUMNS}
                         ,nrows = n_rows, dtype={"fullVisitorId": "str"}
                        )
    
    df = load_jsons_as_cols(df,JSON_COLUMNS)
    
    #while ((len(find_dicts(df,target)) > 0) | (len(find_lists(df,target)) > 0)):

    if len(find_lists(df,target)) > 0:
        for col in find_lists(df,target):
            df[col] = unpack(df[col])
            
    while len(find_dicts(df,target)) > 0:
        df = unfold(df, find_dicts(df,target),target)
        
    '''summa=0
    new_col = []
    #ciclo le colonne che ho scoperto contenere liste
    for col in find_lists(df,target):
        #ciclo le righe che contenti liste
        for row in df[col]:
            #ciclo la lista contenete dizionari
            if len(row)>0:
                #print(row[:3],len(row))
                for d in row:
                    try:
                        summa += float(d['productPrice'])
                    except:
                        pass
                new_col += [summa]
                summa = 0
    df["transactionRevenue"] = pd.Series(new_col)'''
    
    df = drop_const(df,target)

    df = drop_uniques(df,target)
    
    print(f"Loaded {os.path.basename(csv_path)}. Shape: {df.shape}")
    
    return df

## Kernel

In [50]:
#eccezioni, non da droppare
target = ["fullVisitorId","totals.totalTransactionRevenue"]
#num cols
nums = ["visitStartTime","totals.totalTransactionRevenue"]
#cat cols
cats = ["trafficSource.adwordsClickInfo.gclId","trafficSource.referralPath","trafficSource.adContent"]

parameters = {
    #numero massimo di valori in una singola colonna per essere flattata, altrimenti drop
    "max_new_feat":5,
    #inviare a kaggle tramite l' API
    "commit":0,
    #lgbm tuning parameters
    "n_leaves" : 512,
    "feature_fraction" : 0.99,
    "bagging_fraction" : 0.99,
    "learn_rate" : 0.004,
    #train rows
    "train_rows" : 10000,
    #test_rows, per submittare deve essere settato a -1
    "test_rows" : 500,
    #il metodo principale è lgbm ma si può testare anche la regressione lineare
    "test_also_lin_reg" : 1,
    #bagging frequency
    "bagging_freq" : 1,
    #transactionRevenue
    "transactionRevenue" : 0,
    #percentuale di dev e val
    "percentage" : 18,
    #grouping_mode_cats
    "grouping_mode_cats" : "mean",
    #score
    "final_score" : -1,
    #minio di alberi per il rf
    "min_child_samples" : -1
}

locals().update(parameters)

In [51]:
if (commit==1) & (test_rows != -1):
    raise Exception("per submittare devi usare tutte le righe del test")

In [52]:
%time train_df = load_df(train_file,train_rows,target)

Loaded train_v2.csv. Shape: (10000, 92)
CPU times: user 22.9 s, sys: 384 ms, total: 23.3 s
Wall time: 23.3 s


In [53]:
%time test_df = load_df(test_file,test_rows,target)

Loaded test_v2.csv. Shape: (500, 76)
CPU times: user 2.64 s, sys: 4 ms, total: 2.65 s
Wall time: 2.65 s


In [54]:
if "totals.totalTransactionRevenue" in test_df.columns:
    test_df = test_df.drop("totals.totalTransactionRevenue",axis=1)

In [55]:
if not transactionRevenue:
    if "totals.transactionRevenue" in test_df.columns:
        test_df = test_df.drop("totals.transactionRevenue",axis=1)

In [56]:
if not transactionRevenue:
    if "totals.transactionRevenue" in train_df.columns:
        train_df = train_df.drop("totals.transactionRevenue",axis=1)

In [57]:
#controllare che nel nuovo test non droppi totalsRevenue perchè c'è solo nel train
train_df,test_df = drop_uncommons(train_df,test_df,target)

nums = find_num_cols(train_df,target,cats,nums)

cats = find_cat_cols(train_df,target,nums)

In [58]:
cc(train_df,test_df,cats,nums)

In [59]:
for col in nums:
    if (col not in target) | (col=="totals.totalTransactionRevenue"):
        train_df[col] = train_df[col].astype(float)
        if col!="totals.totalTransactionRevenue":
            test_df[col] = test_df[col].astype(float)

In [60]:
# Impute 0 for missing target values
train_df.fillna(0,inplace=True)
test_df.fillna(0,inplace=True)

In [61]:
train_df = stringify_cats(train_df,cats)
test_df = stringify_cats(test_df,cats)

In [62]:
#droppo le colonne che hanno troppa varianza
train_df,test_df,cats = drop_exceeding(train_df,test_df,max_new_feat,cats,target)

In [63]:
cc(train_df,test_df,cats,nums)

In [64]:
train_df[cats] = train_df[cats].astype(str)
test_df[cats] = test_df[cats].astype(str)

In [65]:
######QUI aggiungo il weekday

#train_df["date"].weekday()

In [83]:
%time train_df,test_df,cats = encode_cats(train_df,test_df,cats)

[**         ]

KeyError: "['trafficSource.adwordsClickInfo.page'\n 'trafficSource.adwordsClickInfo.adNetworkType'\n 'trafficSource.adwordsClickInfo.slot'\n 'trafficSource.adwordsClickInfo.isVideoAd'] not in index"

In [67]:
cc(train_df,test_df,cats,nums)

In [68]:
%time train_df = group_me(train_df,"fullVisitorId",cats,nums,grouping_mode_cats)

[***   ]

DataError: No numeric types to aggregate

In [69]:
%time test_df = group_me(test_df,"fullVisitorId",cats,nums,grouping_mode_cats)

[***   ]

DataError: No numeric types to aggregate

In [70]:
cc(train_df,test_df,cats,nums)

In [71]:
load = -1
base = "./saved_conf/"

if load == 1:
    print("hai scelto di importare il dataset da disco")
    train_df = pd.read_csv(base + "dump_train")
    test_df = pd.read_csv(base + "dump_test")
    with open(base + "dump_parameters", 'r') as file:
        file.read(json.loads(parameters))
    locals().update(parameters)
else:
    if load == 0:
        print("hai scelto di scrivere il dataset su disco")
        %time train_df.to_csv(path_or_buf=base + "dump_train", header=True, mode='w',index=False)
        test_df.to_csv(path_or_buf=base + "dump_test", header=True, mode='w',index=False)
        with open(base + "dump_parameters", 'w') as file:
            file.write(json.dumps(parameters))
    else:
        print("hai scelto di non caricare nè scaricare il dataset")

hai scelto di non caricare nè scaricare il dataset


In [72]:
train_id = train_df["fullVisitorId"].values
test_id = test_df["fullVisitorId"].values

In [73]:
#pulizia delle colonne con nomi assurdi
#questa operazione può essere fatta in maniera safe perchè 
#a questo punto i due datasets hanno le stesse colonne con gli stessi nomi

In [74]:
train_df.columns = [col[:30] for col in train_df.columns]
test_df.columns = [col[:30] for col in test_df.columns]

In [75]:
common_feats = list((set(train_df.columns).intersection(set(test_df.columns))).difference(set(target)))

In [76]:
#qui viene bloccato il controllo di coerenza poichè le colonne cambiano, in particolare i nomi 
#vengono accorciati ma è safe farlo perchè i nomi delle colonne sono importanti solo nel preprocessing

In [77]:
#cc(train_df,test_df,cats,nums)

In [78]:
#train_df.head()

In [79]:
#splitto il dataframe in development e validation ma cercando di mantenere in maniera 
#corretta il rapporto dei compratori che è circa dell' 1%

#posso fare confronti con 0 perchè prima tutte le colonne sono state messe a 0 perciò non rischio NA

#divido il train in 2 parti: quelli che hanno speso che sono l' 1% e quelli che non hanno 
#speso 99% e da ognuno estraggo il tot% quindi mantengo il rapporto tra i due
train_money = train_df[train_df["totals.totalTransactionRevenue"]>0]
train_no_money = train_df[train_df["totals.totalTransactionRevenue"]==0]

percent = int(len(train_money)*percentage/100)
train_money_val = train_money.iloc[:percent,]

dev_df = train_money.iloc[percent:len(train_money),]
val_df = train_money_val

percent = int(len(train_no_money)*percentage/100)
train_no_money_val = train_no_money.iloc[:percent,]

dev_df = dev_df.append(train_no_money.iloc[percent:len(train_no_money),])
val_df = val_df.append(train_no_money_val )


###############################################
#voglio lavorare su un subset perciò provo a ridurre la grandezza
#mantengo il rapporto ma perdo info nelle features
#quantity=1

#dev_df=dev_df.iloc[:int(len(dev_df)*quantity),:]
#val_df=val_df.iloc[:int(len(val_df)*quantity),:]
###############################################

#dev_y contiene la colonna addestramento in dev già log1p
dev_y = np.log1p(dev_df["totals.totalTransactionRevenue"].values)
#val_y contiene la colonna target in val già log1p
val_y = np.log1p(val_df["totals.totalTransactionRevenue"].values)

#dev_x contiene colonne numeriche e cat senza transRev
dev_X = dev_df[ common_feats ]
#val_x contiene colonne numeriche e cat senza transRev
val_X = val_df[ common_feats ]
#test è ciò che dobbiamo trovare
test_X = test_df[ common_feats ] 

In [80]:
def write(tipo):
    parameters["final_score"] = final_score
    try:
        if len(pd.read_csv("./tests.csv").columns) != len(parameters.keys())+1:
            print("il file tests.csv contiene meno colonne del necessario, verrà sostituito con perdita")
            !rm "./tests.csv"
    except:
        print("il file tests.csv verrà creato ora perchè non esistente")
    with open("./tests.csv",'a') as ff:
        if os.fstat(ff.fileno()).st_size == 0:
            for k in parameters.keys():
                print(k+',',file = ff,sep='',end='' )
            print("type",file = ff) 
        for v in parameters.values():
            print(str(v)+',',file = ff,sep='',end='' )
        print(tipo,file = ff ) 

## Linear Regression

In [82]:
if test_also_lin_reg == 1:
    pred_test = lin(dev_X,dev_y,test_X)
    pred_val = lin(dev_X,dev_y,val_X)

    val_pred_df = pd.DataFrame({"fullVisitorId":val_df["fullVisitorId"].values})
    val_pred_df["totals.totalTransactionRevenue"] = val_df["totals.totalTransactionRevenue"].values
    val_pred_df["PredictedRevenue"] = np.expm1(pred_val)
    val_pred_df = val_pred_df.groupby("fullVisitorId")["totals.totalTransactionRevenue", "PredictedRevenue"].sum().reset_index()
    val_pred_df[val_pred_df["PredictedRevenue"]>10^20]=0

    final_score = np.sqrt(metrics.mean_squared_error(np.log1p(val_pred_df["totals.totalTransactionRevenue"].values)
                                                     , np.log1p(val_pred_df["PredictedRevenue"].values)))
    print(final_score)
    write("lin_reg")

ValueError: could not convert string to float: '(entrance)'

## LightGBM single-tree

In [ ]:
# custom function to run light-gbm model
def lgbm(train_X, train_y, val_X, val_y, test_X):
    
    params = {
        "objective" : "regression",
        "metric" : "rmse",
        "num_leaves" : n_leaves,
        "feature_fraction" : feature_fraction,
        "bagging_fraction" : bagging_fraction,
        "bagging_freq":bagging_freq,
        "learning_rate" : learn_rate,
        "verbosity" : -1
    }
    
    lgtrain = lgb.Dataset(train_X, label=train_y)
    lgval = lgb.Dataset(val_X, label=val_y)
    model = lgb.train(params, lgtrain, 10000, valid_sets=[lgval], early_stopping_rounds=300, verbose_eval=300,keep_training_booster = True )
    
    pred_test_y = model.predict(test_X, num_iteration=model.best_iteration)
    pred_val_y = model.predict(val_X, num_iteration=model.best_iteration)
    return pred_test_y, model, pred_val_y

In [ ]:
#%time pred_test, model, pred_val = lgbm(dev_X, dev_y, val_X, val_y, test_X)

In [ ]:
def score():
    pred_val[pred_val<0] = 0
    val_pred_df = pd.DataFrame({"fullVisitorId":val_df["fullVisitorId"].values})
    val_pred_df["totals.totalTransactionRevenue"] = val_df["totals.totalTransactionRevenue"].values
    val_pred_df["PredictedRevenue"] = np.expm1(pred_val)
    val_pred_df = val_pred_df.groupby("fullVisitorId")["totals.totalTransactionRevenue", "PredictedRevenue"].sum().reset_index()
    final_score = np.sqrt(metrics.mean_squared_error(np.log1p(val_pred_df["totals.totalTransactionRevenue"].values), np.log1p(val_pred_df["PredictedRevenue"].values)))
    print(final_score)

    sub_df = pd.DataFrame({"fullVisitorId":test_id})
    pred_test[pred_test<0] = 0
    sub_df["PredictedLogRevenue"] = np.expm1(pred_test)
    sub_df = sub_df.groupby("fullVisitorId")["PredictedLogRevenue"].sum().reset_index()
    sub_df.columns = ["fullVisitorId", "PredictedLogRevenue"]
    sub_df["PredictedLogRevenue"] = np.log1p(sub_df["PredictedLogRevenue"])
    
    
    write("LightGBM")
    
    if commit:
        !kaggle competitions submit -c ga-customer-revenue-prediction -f {my_submission_file} -m "No Message"
    
    
    return final_score,sub_df

In [ ]:
def write_df(sub_df):
    sub_df.to_csv(path_or_buf=my_submission_file, header=True, mode='w',index=False)
    !wc -l {my_submission_file}

In [ ]:
#final_score,sub_df = score()

In [ ]:
#write_df(sub_df)

In [ ]:
def plot_imp(model):
    fig, ax = plt.subplots(figsize=(12,18))
    lgb.plot_importance(model, max_num_features=50, height=0.8, ax=ax)
    ax.grid(False)
    plt.title("LightGBM - Feature Importance", fontsize=15)
    plt.show()

In [ ]:
#plot_imp(model)

## LightGBM con rf

In [ ]:
parameters["n_leaves"] = 400
parameters["bagging_fraction"] = 0.99
parameters["feature_fraction"] = 0.99
parameters["bagging_freq"] = 20
parameters["min_child_samples"] = 10
locals().update(parameters)

In [ ]:
# custom function to run light-gbm model
def lgbm_rf(train_X, train_y, val_X, val_y, test_X):
    
    params = {
        "objective" : "regression", 
        "metric" : "rmse",
        "num_leaves" : n_leaves,
        "learning_rate" : learn_rate,
        "bagging_fraction" : bagging_fraction,
        "feature_fraction" : feature_fraction,
        "bagging_freq":bagging_freq,
        'max_depth':-1,
        "min_child_samples" : min_child_samples,
        "boosting":"rf"
    }
    
    lgtrain = lgb.Dataset(train_X, label=train_y)
    lgval = lgb.Dataset(val_X, label=val_y)
    model = lgb.train(params, lgtrain, 3000, valid_sets=[lgval], verbose_eval=500,keep_training_booster = True )
    
    pred_test_y = model.predict(test_X, num_iteration=model.best_iteration)
    pred_val_y = model.predict(val_X, num_iteration=model.best_iteration)
    return pred_test_y, model, pred_val_y

In [ ]:
#%time pred_test, model, pred_val = lgbm_rf(dev_X, dev_y, val_X, val_y, test_X)

In [ ]:
#final_score,sub_df = score()

In [ ]:
#plot_imp(model)

## Iterative Testing

In [ ]:
def test_leaves_lgbm():
    for x in [x for x in range(4,10)]:
        x = 2**x
        print("testing for ",x)
        parameters["n_leaves"] = x
        locals().update(parameters)
        %time pred_test, model, pred_val = lgbm(dev_X, dev_y, val_X, val_y, test_X)
        final_score,sub_df = score()
        print("---------------")

In [ ]:
%time test_leaves_lgbm()

In [ ]:
def test_leaves_lgbm_rf():   
    for x in [2**x for x in range(4,10)]:
        print("testing for ",x)
        parameters["n_leaves"] = x
        locals().update(parameters)
        %time pred_test, model, pred_val = lgbm_rf(dev_X, dev_y, val_X, val_y, test_X)
        final_score,sub_df = score()
        print("---------------")

In [ ]:
%time test_leaves_lgbm_rf()

In [ ]:
def test_leaves_lgbm_best_value():   
    for x in [x for x in range(16,33)]:
        print("testing for ",x)
        parameters["n_leaves"] = x
        locals().update(parameters)
        %time pred_test, model, pred_val = lgbm(dev_X, dev_y, val_X, val_y, test_X)
        final_score,sub_df = score()
        print("---------------")

In [ ]:
%time test_leaves_lgbm_best_value()